<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir spacy_ner_data

In [12]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/40_resumes_annotated.json', "r") as f:
    data = json.load(f)

def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # Only add if there's no overlap with the previous entity
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        entities = remove_overlapping_entities(entities)  # Remove overlapping entities
        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [7]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [13]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
# train model using hyperparameters set in config.cfg
# trained model in output/ dir
!python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0005
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    238.65    0.00    0.00    0.00    0.00
  3     100       2129.88   9813.94   18.94   21.25   17.09    0.19
  6     200        433.86   5971.17   27.32   32.56   23.53    0.27
  9     300        174.71   4187.37   32.45   28.57   37.54    0.32
 12     400        655.81   3933.13   30.71   26.71   36.13    0.31
 15     500        834.10   2505.04   27.80   19.36   49.30    0.28
 18     600        112.18   1961.11   31.86   27.24   38.38    0.32
 21     700         97.26   1453.16   34.12   30.72   38.38    0.34
 25     800        109.06   12

In [17]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate output/model-best spacy_ner_data/test_data.spacy -dp output

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   30.00 
NER R   40.34 
NER F   34.41 
SPEED   2476  


=============================== NER (per type) ===============================

                 P        R        F
NAME        100.00    62.50    76.92
PHONE       100.00   100.00   100.00
SKILL        24.81    36.84    29.65
WORK PER     48.39    75.00    58.82
COMPANY      14.29     5.88     8.33
JOB          53.85    33.33    41.18
STUDY PER    50.00    50.00    50.00
DEG          33.33    50.00    40.00
UNI          55.56    71.43    62.50
LOC         100.00    75.00    85.71

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__main__.py

In [18]:
# make prediction
import spacy
resume_text = 'dot net developer robert smith phone 123 456 78 99 email infoqwikresumecom website wwwqwikresumecom linkedin linkedincomqwikresume address 1737 marshville road alabama objective dot net developer seven years experience design development webbased windows based applications using net technologies hands experience phases software development life cycle sdlc like requirement gathering analysis architectural detail design documentation development testing implementation using agile methodologies like scrum xp test driven environment skills programming languages net technologies c sql plsql web forms win forms web technologies scripting html dhtml css xmlangular jsbootstrap ajax toolkit jquery javascript telerikkendo ui database ms sql server operating systems windows 8 packages ms office amp visio ms frontpage iis version control tools git hub vss tfs methodologies agile oops scrum soa reporting crystal reports net ms sql server reporting services ssrs work experience dot net developer charter communications june 2015 present assisting developing architectural design functional specifications involving analysis designing coding implementation application developing dynamic web page implemented creatively implemented design requirements using client side scripting language technologies assisting agile software development management activities respond unpredictability iterative sprints designing developing web application migrating project mvc architecture using mvc 3 separate internal representations information involved developing telerik kendo ul controls building application enabling focus value generating development tasks involving development presentation logic gui aspnet pages dot net developer abc corp 2011 2015 coded updated maintained computer programs assisted developers prepare high level technical design documents performed code enhancements assist performance analysis provided technical guidance programming standards team trained users team coordinated client amp offshore tearn meet project objectives participated backlog grooming meeting work client remove barriers team worked individual developer also manage offshore team free resume template copyright qwikresumecom usage guidelines'
nlp = spacy.load("output/model-best")
doc = nlp(resume_text)

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

(robert smith, 123 456 78 99, infoqwikresumecom, alabama, agile, scrum, c, sql, plsql, scripting, html, css, jquery, ui, database, ms sql, windows, git, tfs, agile, oops, scrum, soa, dot net developer, june 2015 present assisting, mvc, mvc, gui, aspnet, abc corp, 2011 2015)
robert smith: NAME
123 456 78 99: PHONE
infoqwikresumecom: EMAIL
alabama: LOC
agile: SKILL
scrum: SKILL
c: SKILL
sql: SKILL
plsql: SKILL
scripting: SKILL
html: SKILL
css: SKILL
jquery: SKILL
ui: SKILL
database: SKILL
ms sql: SKILL
windows: SKILL
git: SKILL
tfs: SKILL
agile: SKILL
oops: SKILL
scrum: SKILL
soa: SKILL
dot net developer: JOB
june 2015 present assisting: WORK PER
mvc: SKILL
mvc: SKILL
gui: SKILL
aspnet: SKILL
abc corp: COMPANY
2011 2015: WORK PER


In [19]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# download trained model

## Flair NER

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.4 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=742bd16ac6d02b9a3c1bdd34f68262c78aa59427d9c2ba9a8ac5cdcacf7b9d6a
  Stored in directory: /root/.cach

In [ ]:
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file)

2024-10-29 14:19:43,450 Reading data from .
2024-10-29 14:19:43,451 Train: flair_train.txt
2024-10-29 14:19:43,454 Dev: None
2024-10-29 14:19:43,455 Test: flair_test.txt
2024-10-29 14:19:43,629 No dev split found. Using 10% (i.e. 2 samples) of the train split as dev data


In [ ]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger

embeddings = StackedEmbeddings([
                WordEmbeddings('glove'),
                WordEmbeddings('en-crawl')
            ])

tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),
                         tag_type='ner',
                         use_crf=True)

2024-10-29 14:20:05,377 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpuoh0v_ah


100%|██████████| 153M/153M [00:06<00:00, 23.8MB/s]

2024-10-29 14:20:12,442 copying /tmp/tmpuoh0v_ah to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2024-10-29 14:20:12,956 removing temp file /tmp/tmpuoh0v_ah
2024-10-29 14:20:13,343 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpefkn2os3


100%|██████████| 20.5M/20.5M [00:01<00:00, 12.2MB/s]

...

2024-10-29 14:21:43,243 removing temp file /tmp/tmpmwi5difw
2024-10-29 14:21:43,941 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M not found in cache, downloading to /tmp/tmphb6pog37


100%|██████████| 37.5M/37.5M [00:02<00:00, 14.3MB/s]

2024-10-29 14:21:47,129 copying /tmp/tmphb6pog37 to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M
2024-10-29 14:21:47,180 removing temp file /tmp/tmphb6pog37


2024-10-29 14:22:04,219 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>


<ipython-input-32-687232df04ba>:12: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),


In [ ]:
# train flair ner model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train('flair_output/',
               learning_rate=0.1,
               mini_batch_size=32,
               max_epochs=10)

# trainer.train(
#      base_path='/flair_output/',
#      learning_rate=0.1,
#      mini_batch_size=32,
#      max_epochs=10,
#      patience=3,
#      embeddings_storage_mode='gpu',
#     #  checkpoint=True,
#      use_amp=True,  # Use mixed precision training
#      train_with_dev=False,
#     #  monitor_train=True,
#     #  monitor_test=True,
#      evaluation_metric=EvaluationMetric.MICRO_F1_SCORE,
# )

trainer.train(
    base_path='/flair_output/',
    learning_rate=0.1,
    mini_batch_size=32,
    max_epochs=10,
    patience=3,
    embeddings_storage_mode='gpu',
    use_amp=True,  # Use mixed precision training
    train_with_dev=False,
    main_evaluation_metric=EvaluationMetric.MICRO_F1_SCORE  # Set the evaluation metric here
)

2024-10-29 14:46:39,551 ----------------------------------------------------------------------------------------------------
2024-10-29 14:46:39,554 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=400, out_features=400, bias=True)
  (rnn): LSTM(400, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-10-29 14:46:39,557 ----------------------------------------------------------------------------------------------------
2024-10-29 14:46:39,562 Corpus: 20 train + 2 dev + 6 test sentences
2024-10-29 14:46:39,565 -------------------------------------

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-10-29 14:48:36,579 epoch 1 - iter 1/1 - loss 0.00000000 - time (sec): 116.98 - samples/sec: 81.64 - lr: 0.100000 - momentum: 0.000000
2024-10-29 14:48:36,587 ----------------------------------------------------------------------------------------------------
2024-10-29 14:48:36,591 EPOCH 1 done: loss 0.0000 - lr: 0.100000


100%|██████████| 1/1 [00:00<00:00,  2.67it/s]

2024-10-29 14:48:36,991 DEV : loss -5.580357083090348e-07 - f1-score (micro avg)  0.0
2024-10-29 14:48:36,997  - 0 epochs without improvement
2024-10-29 14:48:37,001 ----------------------------------------------------------------------------------------------------


...

2024-10-29 15:06:35,564 ----------------------------------------------------------------------------------------------------
2024-10-29 15:06:35,570 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=400, out_features=400, bias=True)
  (rnn): LSTM(400, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-10-29 15:06:35,573 ----------------------------------------------------------------------------------------------------
2024-10-29 15:06:35,576 Corpus: 20 train + 2 dev + 6 test sentences
2024-10-29 15:06:35,579 -------------------------------------

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-10-29 15:08:01,262 epoch 1 - iter 1/1 - loss -0.00001145 - time (sec): 85.65 - samples/sec: 111.51 - lr: 0.100000 - momentum: 0.000000
2024-10-29 15:08:01,271 ----------------------------------------------------------------------------------------------------
2024-10-29 15:08:01,274 EPOCH 1 done: loss -0.0000 - lr: 0.100000


100%|██████████| 1/1 [00:00<00:00,  2.67it/s]


TypeError: 'EvaluationMetric' object is not subscriptable

In [ ]:
# evaluate model
result, score = trainer.evaluate(corpus.test)
print(result)
print(score)

AttributeError: 'ModelTrainer' object has no attribute 'evaluate'

In [ ]:
# make prediction
import flair
model = SequenceTagger.load('flair_output/final-model.pt')
sentence = flair.data.Sentence(resume_text)

model.predict(sentence)

print(sentence.to_tagged_string())

2024-10-29 15:09:34,915 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>
Sentence[278]: "dot net developer robert smith phone 123 456 78 99 email infoqwikresumecom website wwwqwikresumecom linkedin linkedincomqwikresume address 1737 marshville road alabama objective dot net developer seven years experience design development webbased windows based applications using net technologies hands experience phases software development life cycle sdlc like requirement gathering analysis architectural detail design documentation development testing implementation using agile methodologies like scrum xp test driven environment skills programming languages net technologies c sql plsql web forms win forms web technologies scripting html dhtml css xmlangular jsbootstrap ajax toolkit jquery javascript telerikkendo ui database ms sql server operating systems windows 8 packages ms office amp visio ms frontpage iis version control tools git hub vss tfs methodologies agile oops scrum s